<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/lang-chain-projects/blob/main/RAG_18april.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### 1. change the prompt to SQL query
###### 2. send the received SQL query to SQL-Query generate model
###### 3. execute the query
###### 4. send to LLM

# install libraries

In [2]:
# !pip install langchain
# !pip install langchain_core
# !pip install langchain-huggingface
# !pip install langchain_experimental
# !pip install langchain-community
# !pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

# import libraries

In [3]:
#langchain
from langchain.prompts import (
    PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
)
from langchain_core.messages import (
    AIMessage,HumanMessage,SystemMessage
)

from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import (
    StructuredOutputParser, ResponseSchema
)

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from langchain import hub
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.prebuilt import create_react_agent

#common imports
import os
from google.colab import userdata
import pandas as pd
import sqlite3

In [4]:
import ast

### set configuration

In [5]:
os.environ['HF_TOKEN']=userdata.get('HUGGINGFACEHUB_ACCESS_TOKEN')

# agent 1

### convert prompt to SQL query using LLM

##### prompt

In [ ]:
example_prompt=[
HumanMessage(content='get me the details of the employee named xyz'),
AIMessage(content='select * from employee_details where employee_name=xyz'),
HumanMessage(content='get me the details of the employee id 12345'),
AIMessage(content='select * from employee_details where employee_id=12345'),
HumanMessage(content='what is the DOF of xyz'),
AIMessage(content= 'select DOF from employee_details where employee_name=xyz')
]

schema=[
    ResponseSchema(name='sql query',description='select * from employee_details'),
    ResponseSchema(name='sql query',description='select * from employee_details where employee_id=456')
]

_parser=StructuredOutputParser.from_response_schemas(schema)

In [ ]:
_prompt=ChatPromptTemplate([
    ('system','you are an helpful SQL Agent who can convert user prompt to SQL query. \n {format_instruction}'),
    MessagesPlaceholder(variable_name='examples'),
    ('human','{question}')
],
input_variables=['question'],
      partial_variables={'format_instruction':_parser.get_format_instructions()}
)

##### model

In [ ]:
_repo_id_01='TinyLlama/TinyLlama-1.1B-Chat-v1.0'
_repo_id_02='deepseek-ai/DeepSeek-V3-0324'
_repo_id_03='deepseek-ai/DeepSeek-R1'
_repo_id_04='google/gemma-2-2b-it'
_repo_id_05='meta-llama/Llama-4-Scout-17B-16E-Instruct'
_repo_id_06='mistralai/Mistral-7B-Instruct-v0.3'

_llm=HuggingFaceEndpoint(
    repo_id=_repo_id_06,
    task='text-generation',
    model_kwargs={
     'max_length':300
    }
)

_model=ChatHuggingFace(llm=_llm)

chain=(_prompt | _model | _parser)

In [ ]:
chain.invoke({
    'examples':example_prompt,
    'question':'what is the salary of senior manager'
})

{'sql query': "SELECT salary FROM employee_details WHERE position = 'senior manager'"}

# agent 2

###### --- notes for agent2
###### 1. select a SQL query model
###### 2. select a database engine
###### 3. create metadata & prompt file, mount it in gdrive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import argparse

def generate_prompt(question, prompt_file="/content/drive/MyDrive/prompt.md", metadata_file="/content/drive/MyDrive/waila_table_meta_data.sql"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    with open(metadata_file, "r") as f:
        table_metadata_string = f.read()

    prompt = prompt.format(
        user_question=question, table_metadata_string=table_metadata_string
    )
    return prompt


def get_tokenizer_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
    return tokenizer, model

def run_inference(question, prompt_file="/content/drive/MyDrive/prompt.md", metadata_file="/content/drive/MyDrive/waila_table_meta_data.sql"):
    tokenizer, model = get_tokenizer_model("defog/sqlcoder-7b-2")
    prompt = generate_prompt(question, prompt_file, metadata_file)

    # make sure the model stops generating at triple ticks
    # eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
    eos_token_id = tokenizer.eos_token_id
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        do_sample=False,
        return_full_text=False, # added return_full_text parameter to prevent splitting issues with prompt
        num_beams=5, # do beam search with 5 beams for high quality results
    )
    generated_query = (
        pipe(
            prompt,
            num_return_sequences=1,
            eos_token_id=eos_token_id,
            pad_token_id=eos_token_id,
        )[0]["generated_text"]
        .split(";")[0]
        .split("```")[0]
        .strip()
        + ";"
    )
    return generated_query


# _default_question="list all employees whose name starts with r"
# parser = argparse.ArgumentParser(description="Run inference on a question")
# parser.add_argument("-q","--question", type=str, default=_default_question, help="Question to run inference on")
# args = parser.parse_args()
# question = args.question
# print("Loading a model and generating a SQL query for answering your question...")
# print(run_inference(question))

In [ ]:
prompt=generate_prompt('how many employees are there in total')
tokenizer, model=get_tokenizer_model('defog/sqlcoder-7b-2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
eos_token_id = tokenizer.eos_token_id
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=False,
    return_full_text=False, # added return_full_text parameter to prevent splitting issues with prompt
    num_beams=5, # do beam search with 5 beams for high quality results
)

Device set to use cuda:0


In [ ]:
generated_query=pipe(
            prompt,
            num_return_sequences=1,
            eos_token_id=eos_token_id,
            pad_token_id=eos_token_id,
        )

OutOfMemoryError: CUDA out of memory. Tried to allocate 68.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 64.12 MiB is free. Process 16405 has 14.68 GiB memory in use. Of the allocated memory 14.04 GiB is allocated by PyTorch, and 518.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# approach 3

##### prompt model

In [6]:
_repo_id_01='TinyLlama/TinyLlama-1.1B-Chat-v1.0'
_repo_id_02='deepseek-ai/DeepSeek-V3-0324'
_repo_id_03='deepseek-ai/DeepSeek-R1'
_repo_id_04='google/gemma-2-2b-it'
_repo_id_05='meta-llama/Llama-4-Scout-17B-16E-Instruct'
_repo_id_06='mistralai/Mistral-7B-Instruct-v0.3'
_repo_id_07='OpenAccessAI/Mistral-7B-OpenFunction-v0.2'
_repo_id_tool_calling='HuggingFaceH4/zephyr-7b-beta'

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
prompt_template

ChatPromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-agent-system-prompt', 'lc_hub_commit_hash': '31156d5fe3945188ee172151b086712d22b8c70f8f1c0505f5457594424ed352'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interact

In [9]:
_data=pd.read_csv('/content/employee-list.csv')
conn = sqlite3.connect("waila_employee.db")
_data.to_sql("employee_table", conn, if_exists="replace", index=False)
db = SQLDatabase.from_uri(
    "sqlite:///waila_employee.db",
    include_tables=['employee_table']
    )

In [10]:
_llm=HuggingFaceEndpoint(
    repo_id=_repo_id_05,
    task='text-generation',
    model_kwargs={
     'max_length':300
    }
)

_model=ChatHuggingFace(llm=_llm)

toolkit = SQLDatabaseToolkit(db=db, llm=_model)

system_message = prompt_template.format(dialect="SQLite", top_k=5)
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(_model, toolkit.get_tools(), prompt=system_message)

##### rough

In [11]:
example_query = "select Name from employee_table where emailid ='aaradhya@gmail.com'"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

# agent_executor.invoke

================================ Human Message =================================

select Name from employee_table where emailid ='aaradhya@gmail.com'


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-682243d3-7ec4855165af96557f9a6fc8;74ce7aa0-cf21-47ac-b2aa-1c9aa4f13160)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

##### rough end

In [86]:
example_prompt=[
HumanMessage(content='get me the details of xyz'),
AIMessage(content='select * from employee_table where name like "%xyz%"'),
HumanMessage(content='who is xyz'),
AIMessage(content='select * from employee_table where name like "%xyz%"'),
HumanMessage(content='get me the details of xyz@gmail.com'),
AIMessage(content='select * from employee_table where emailId="xyz@gmail.com"'),
HumanMessage(content='where is xyz from'),
AIMessage(content= 'select country from employee_table where name="xyz"'),
HumanMessage(content='when did xyz join the company'),
AIMessage(content= 'select HiredDate from employee_table where name="xyz"'),
HumanMessage(content='how many employees does the company have'),
AIMessage(content= 'select count(*) from employee_table'),
]

schema=[
    ResponseSchema(name='sql query',description='select * from employee_table'),
    ResponseSchema(name='sql query',description='select * from employee_table where Country="Uganda"')
]

_parser=StructuredOutputParser.from_response_schemas(schema)

In [101]:
_parser.get_format_instructions()

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sql query": string  // select * from employee_table\n\t"sql query": string  // select * from employee_table where Country="Uganda"\n}\n```'

In [87]:
_prompt=ChatPromptTemplate([
    ('system','you are an helpful SQL Agent who can convert user prompt to SQL query. \n {format_instruction}'),
    MessagesPlaceholder(variable_name='examples'),
    ('human','{question}')
],
input_variables=['question'],
      partial_variables={'format_instruction':_parser.get_format_instructions()}
)

In [11]:
chain=(
    _prompt | _model | _parser
)

In [15]:
result=chain.invoke({
    'examples':example_prompt,
    'question':'get me details of aaradhya@gmail.com'
})

In [16]:
result

{'sql query': "select * from employee_table where emailId='aaradhya@gmail.com'"}

In [19]:
final_answer=db.run(result['sql query'])

In [49]:
result=ast.literal_eval(final_answer)

In [73]:
' '.join([str(r) for r in result[0]])

'aaradhya@gmail.com Anjana Raj WA7898123 Rwanda None BDO Africa Holdings Ltd Technologies, Media and Telecom None None Development None develop P2 None None Developer 13-05-2024 13-05-2024 yes Guhan 91451198306.0 Akass Akash Ajay S Anj Anj'

##### retriever model

In [18]:
_retriever_model=ChatHuggingFace(llm=_llm)

In [79]:
prompt=PromptTemplate(
    template='''
    you are an helpful assistant, please answer the {question} from the following {context}
    ''',
    input_variables=['question','context']
)

In [80]:
formatted_prompt = prompt.format(
    question='get me details of aaradhya@gmail.com',
    context='aaradhya@gmail.com Anjana Raj WA7898123 Rwanda None BDO Africa Holdings Ltd Technologies, Media and Telecom None None Development None develop P2 None None Developer 13-05-2024 13-05-2024 yes Guhan 91451198306.0 Akass Akash Ajay S Anj Anj'
)

# Step 3: Pass the string into the model (assuming _retriever_model is an LLM or chain)
answer02 = _retriever_model.invoke(formatted_prompt)

In [82]:
answer02.content

'Here is the information extracted from the provided data:\n\n- Email Address: aaradhya@gmail.com\n- Full Name: Anjana Raj\n- Phone Number: WA7898123\n- Country: Rwanda\n- Company Name: BDO Africa Holdings Ltd\n- Department: Technologies, Media, and Telecom\n- Job Title / Role: None specified\n- Current Position: None specified\n- Specialization: Development\n- Experience Starting Date: None specified\n- Experience End Date: None specified\n- Notice Period: None specified\n- Emergency Contact: Guhan\n- Emergency Contact Phone Number: 91451198306\n- Reporting Manager: Not specified in the provided data\n- Alternate Names: Akass Akash, Ajay S, Anj, Anj (No information about their relationship to the main profile is provided)'